In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_excel("all_issues_sampled_v2.xlsx")

# Step 1: Create a subset of the dataset separating 'No miscommunication' from the rest
miscommunication_df = df[df['Types of miscommunication'] != 'No miscommunication']

# Step 2: Remove 'Duplicate issue' from the subset
non_duplicate_df = miscommunication_df[miscommunication_df['Root cause'] != 'Others']

# Step 3: Group by 'Types of miscommunication', 'Root cause', and 'Timing' and calculate timing counts
timing_analysis = non_duplicate_df.groupby(['Types of miscommunication', 'Root cause', 'Timing']).size().reset_index(name='Count')

# Calculate the total number of issues for each combination
total_counts = timing_analysis.groupby(['Types of miscommunication', 'Root cause'])['Count'].sum().reset_index(name='Total')

# Merge total counts back to timing_analysis
timing_analysis = pd.merge(timing_analysis, total_counts, on=['Types of miscommunication', 'Root cause'])

# Calculate the percentage for each timing category correctly
timing_analysis['Timing Percentage'] = (timing_analysis['Count'] / timing_analysis['Total']) * 100

# Pivot the timing information for better readability
timing_pivot = timing_analysis.pivot_table(
    index=['Types of miscommunication', 'Root cause'],
    columns='Timing',
    values=['Count', 'Timing Percentage'],
    fill_value=0
).reset_index()

# Re-create the combined counts and percentage DataFrame
non_duplicate_grouped = non_duplicate_df.groupby(['Types of miscommunication', 'Root cause']).size().reset_index(name='Count')
non_duplicate_grouped['Percentage (%)'] = (non_duplicate_grouped['Count'] / non_duplicate_grouped['Count'].sum()) * 100

# Pivot the counts and percentages
non_duplicate_pivot = non_duplicate_grouped.pivot(index='Types of miscommunication', columns='Root cause', values='Count').fillna(0)
non_duplicate_pivot_percentage = non_duplicate_grouped.pivot(index='Types of miscommunication', columns='Root cause', values='Percentage (%)').fillna(0)

# Combine counts and percentages
#combined_non_duplicate_df = non_duplicate_pivot.astype(int).astype(str) + " (" + non_duplicate_pivot_percentage.round(2).astype(str) + "%)"
combined_non_duplicate_df = non_duplicate_pivot.astype(int).astype(str)

# Ensure correct mapping of timing info
for index, row in timing_pivot.iterrows():
    # Retrieve the current value from the combined DataFrame
    current_value = combined_non_duplicate_df.loc[row['Types of miscommunication'], row['Root cause']]
    
    # Construct the timing string with both count and percentage
    timing_info = (
        f"\nDuring discussion: {row[('Count', 'During discussion')]} ({row[('Timing Percentage', 'During discussion')]:.2f}%)\n"
        #f"Follow-up discussions: {row[('Count', 'In Follow-Up Discussion')]} ({row[('Timing Percentage', 'In Follow-Up Discussion')]:.2f}%)\n"
        f"Post-merge: {row[('Count', 'Post-merge')]} ({row[('Timing Percentage', 'Post-merge')]:.2f}%)"
    )
    
    # Combine the current value with the timing information
    combined_non_duplicate_df.loc[row['Types of miscommunication'], row['Root cause']] = current_value + timing_info






# Display the final table with proper formatting
formatted_table = combined_non_duplicate_df.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
    'vertical-align': 'top'
})

formatted_table
